In [7]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
import os
import numpy as np

# --- Configuration ---
MODEL_NAME = "distilbert-base-uncased"
TRAIN_FILE = "/kaggle/input/fine-tune3/train_prompts.csv"
VALIDATION_FILE = "/kaggle/input/fine-tune3/validation_prompts.csv"
TEST_FILE = "/kaggle/input/fine-tune3/test_prompts.csv"
MAX_LENGTH = 128
SAVE_PATH = "./prompt_injection_distilbert_v3"
EPOCHS = 3
BATCH_SIZE = 16
LEARNING_RATE = 5e-5

In [8]:
# ==============================================================================
# ✅ Step 1: Check Hardware and Setup
# ==============================================================================
print("--- 1. Checking Hardware Setup ---")
try:
    # Set up GPU or TPU strategy if available (common in Kaggle)
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU:", tpu.master())
except ValueError:
    strategy = tf.distribute.get_strategy()
    print("Running on default strategy (likely CPU or GPU)")

print("Number of replicas (cores/devices):", strategy.num_replicas_in_sync)

--- 1. Checking Hardware Setup ---
Running on default strategy (likely CPU or GPU)
Number of replicas (cores/devices): 1


In [9]:
# ==============================================================================
# ✅ Step 2: Load Data
# ==============================================================================
print("\n--- 2. Loading Data ---")

if not all(os.path.exists(f) for f in [TRAIN_FILE, VALIDATION_FILE, TEST_FILE]):
    print(f"Error: One or more data files ({TRAIN_FILE}, {VALIDATION_FILE}, {TEST_FILE}) not found.")
    print("Please ensure you have created and saved these files first.")
    exit()

try:
    df_train = pd.read_csv(TRAIN_FILE)
    df_val = pd.read_csv(VALIDATION_FILE)
    df_test = pd.read_csv(TEST_FILE)
    
    # Extract text and labels
    train_texts = df_train['prompt']
    train_labels = df_train['label']
    
    val_texts = df_val['prompt']
    val_labels = df_val['label']
    
    test_texts = df_test['prompt']
    test_labels = df_test['label']
    
    print(f"Loaded Train: {len(train_texts)} samples | Val: {len(val_texts)} | Test: {len(test_texts)}")

except Exception as e:
    print(f"Error loading data: {e}")
    exit()




--- 2. Loading Data ---
Loaded Train: 209134 samples | Val: 26142 | Test: 26142


In [10]:
# ==============================================================================
# ✅ Step 3: Tokenizer
# ==============================================================================
print("\n--- 3. Tokenizing Data ---")

# Use DistilBertTokenizerFast for better performance
tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(
    list(train_texts.astype(str)), 
    truncation=True, 
    padding='max_length', 
    max_length=MAX_LENGTH, 
    return_tensors="tf"
)
val_encodings = tokenizer(
    list(val_texts.astype(str)), 
    truncation=True, 
    padding='max_length', 
    max_length=MAX_LENGTH, 
    return_tensors="tf"
)
test_encodings = tokenizer( 
    list(test_texts.astype(str)), 
    truncation=True, 
    padding='max_length', 
    max_length=MAX_LENGTH, 
    return_tensors="tf"
)

print(f"Tokenization complete (Max Length: {MAX_LENGTH}).")



--- 3. Tokenizing Data ---
Tokenization complete (Max Length: 128).


In [11]:
# ==============================================================================
# ✅ Step 4: Model Initialization and Compile (within Strategy scope)
# ==============================================================================
with strategy.scope():
    print("\n--- 4. Initializing and Compiling Model ---")

    # Load the DistilBERT model configured for sequence classification (2 labels: 0 or 1)
    model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

    # Define Optimizer, Loss, and Metrics
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    
    # Use SparseCategoricalCrossentropy since labels are integers (0 or 1)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    # Use SparseCategoricalAccuracy for the same reason
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy("accuracy")]

    # Compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    print("Model compilation successful.")



--- 4. Initializing and Compiling Model ---


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Model compilation successful.


In [12]:
# ==============================================================================
# ✅ Step 5: Train the model
# ==============================================================================
print("\n--- 5. Starting Training ---")
history = model.fit(
    x=dict(train_encodings),
    y=train_labels.values,
    batch_size=BATCH_SIZE * strategy.num_replicas_in_sync, # Adjust batch size for multi-device training
    epochs=EPOCHS,
    validation_data=(dict(val_encodings), val_labels.values)
)
print("Training Complete.")


--- 5. Starting Training ---
Epoch 1/3


I0000 00:00:1759442369.200674     101 service.cc:148] XLA service 0x78252c58a850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759442369.201326     101 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1759442369.286726     101 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1759442369.446748     101 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13071/13071 [==============================] - 1829s 137ms/step - loss: 0.0579 - accuracy: 0.9798 - val_loss: 0.0865 - val_accuracy: 0.9723
Epoch 2/3
13071/13071 [==============================] - 1774s 136ms/step - loss: 0.0292 - accuracy: 0.9903 - val_loss: 0.0413 - val_accuracy: 0.9856
Epoch 3/3
13071/13071 [==============================] - 1769s 135ms/step - loss: 0.0277 - accuracy: 0.9913 - val_loss: 0.0572 - val_accuracy: 0.9847
Training Complete.


In [13]:
# ==============================================================================
# ✅ Step 6: Evaluate the model (FINAL evaluation on the Test set)
# ==============================================================================
print("\n--- 6. Final Evaluation on UNSEEN Test Set ---")
loss, accuracy = model.evaluate(dict(test_encodings), test_labels.values, batch_size=BATCH_SIZE)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print("-" * 50)

# Optional: Generate more detailed metrics like Precision, Recall, F1-Score
test_predictions = model.predict(dict(test_encodings)).logits
test_predictions = np.argmax(test_predictions, axis=1)

from sklearn.metrics import classification_report
print("\n--- Detailed Classification Report ---")
print(classification_report(test_labels, test_predictions, target_names=["Benign (0)", "Malicious (1)"]))




--- 6. Final Evaluation on UNSEEN Test Set ---
1634/1634 [==============================] - 79s 48ms/step - loss: 0.0576 - accuracy: 0.9853
Test Loss: 0.0576
Test Accuracy: 0.9853
--------------------------------------------------
817/817 [==============================] - 69s 82ms/step

--- Detailed Classification Report ---
               precision    recall  f1-score   support

   Benign (0)       0.98      0.99      0.99     13071
Malicious (1)       0.99      0.98      0.99     13071

     accuracy                           0.99     26142
    macro avg       0.99      0.99      0.99     26142
 weighted avg       0.99      0.99      0.99     26142



In [14]:
# ==============================================================================
# ✅ Step 7: Save the model and tokenizer
# ==============================================================================
print("\n--- 7. Saving Model and Tokenizer ---")
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print(f"Model and Tokenizer successfully saved to '{SAVE_PATH}'")
print("Don't forget to download this folder!")



--- 7. Saving Model and Tokenizer ---
Model and Tokenizer successfully saved to './prompt_injection_distilbert_v3'
Don't forget to download this folder!
